# 02. First Steps with LangChain

## 安装依赖

In [1]:
%uv pip install langchain~=0.3 langchain-core~=0.3 langchain-community~=0.3 langchain-openai~=0.3 langgraph~=0.6

Resolved 59 packages in 101ms                                        
░░░░░░░░░░░░░░░░░░░░ [0/6] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 6 packages in 19ms                                
 + langgraph==0.6.11
 + langgraph-checkpoint==3.0.0
 + langgraph-prebuilt==0.6.5
 + langgraph-sdk==0.2.9
 + ormsgpack==1.11.0
 + xxhash==3.6.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
%uv pip install python-dotenv~=1.1

Audited 1 package in 1ms
Note: you may need to restart the kernel to use updated packages.


工具类

In [3]:
import os

import dotenv
from langchain_openai import ChatOpenAI


class Config:
    def __init__(self):
        # By default, load_dotenv doesn't override existing environment variables and looks for a .env file in same directory as python script or searches for it incrementally higher up.
        dotenv_path = dotenv.find_dotenv(usecwd=True)
        if not dotenv_path:
            raise ValueError("No .env file found")
        dotenv.load_dotenv(dotenv_path=dotenv_path)

        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OPENAI_API_KEY is not set")

        base_url = os.getenv("OPENAI_API_BASE_URL")
        if not base_url:
            raise ValueError("OPENAI_API_BASE_URL is not set")

        model = os.getenv("OPENAI_MODEL")
        if not model:
            raise ValueError("OPENAI_MODEL is not set")

        vl_model = os.getenv("OPENAI_VL_MODEL")

        self.api_key = api_key
        self.base_url = base_url
        self.model = model
        self.vl_model = vl_model

    def new_openai_like(self, **kwargs) -> ChatOpenAI:
        # 参考：https://bailian.console.aliyun.com/?tab=api#/api/?type=model&url=2587654
        # 参考：https://help.aliyun.com/zh/model-studio/models
        # ChatOpenAI 文档参考：https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI
        return ChatOpenAI(
            api_key=self.api_key, base_url=self.base_url, model=self.model, **kwargs
        )

    def new_openai_like_vl(self, **kwargs) -> ChatOpenAI:
        if not self.vl_model:
            raise ValueError("OPENAI_VL_MODEL is not set")

        # 参考：https://bailian.console.aliyun.com/?tab=api#/api/?type=model&url=2587654
        # 参考：https://help.aliyun.com/zh/model-studio/models
        # ChatOpenAI 文档参考：https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html#langchain_openai.chat_models.base.ChatOpenAI
        return ChatOpenAI(
            api_key=self.api_key, base_url=self.base_url, model=self.vl_model, **kwargs
        )

## Exploring LangChain’s building blocks
### Model interfaces
#### LLM interaction patterns

In [ ]:
# initialize OpenAI-like model
llm = Config().new_openai_like()

# Both can be used with the same interface
response = llm.invoke("Tell me a joke about light bulbs!")
print(response.content)

Sure! Here's a classic light bulb joke for you:

**How many programmers does it take to change a light bulb?**  
*None — It’s a hardware problem.*

😄

And here’s another one for variety:

**How many psychologists does it take to change a light bulb?**  
*Just one — but the light bulb has to really want to change.*

Let me know if you want a joke about a specific type of bulb — LED, fluorescent, or even a 19th-century gas lamp!


#### Development testing

In [5]:
from langchain_community.llms import FakeListLLM

# Create a fake LLM that always returns the same responses
fake_llm = FakeListLLM(responses=["Hello"])

result = fake_llm.invoke("Any input will return Hello")
print(result)  # Output: Hello

Hello


#### Working with chat models

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

# initialize OpenAI-like model
llm = Config().new_openai_like()

messages = [
    SystemMessage(content="You're a helpful programming assistant"),
    HumanMessage(content="Write a Python function to calculate factorial"),
]
response = llm.invoke(messages)
print(response.content)

Here's a Python function to calculate the factorial of a number:

```python
def factorial(n):
    """
    Calculate the factorial of a non-negative integer n.
    
    Args:
        n (int): A non-negative integer
        
    Returns:
        int: The factorial of n (n!)
        
    Raises:
        ValueError: If n is negative
        TypeError: If n is not an integer
    """
    # Input validation
    if not isinstance(n, int):
        raise TypeError("Factorial is only defined for integers")
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    
    # Base cases
    if n == 0 or n == 1:
        return 1
    
    # Calculate factorial iteratively
    result = 1
    for i in range(2, n + 1):
        result *= i
    
    return result
```

Here's also a recursive version for comparison:

```python
def factorial_recursive(n):
    """
    Calculate the factorial of a non-negative integer n using recursion.
    
    Args:
        n (int): A non-negat

#### Reasoning models

In [8]:
from langchain_core.prompts import ChatPromptTemplate


# initialize OpenAI-like model with reasoning_effort parameter
# llm = Config().new_openai_like(reasoning_effort="high")
llm = Config().new_openai_like(
    reasoning_effort="high"  # Options: "low", "medium", "high"
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an experienced programmer and mathematical analyst."),
        ("user", "{problem}"),
    ]
)

# Initialize with reasoning_effort parameter
# chat = ChatOpenAI(
# model="o3-mini","
# reasoning_effort="high" # Options: "low", "medium", "high"
# )
chain = template | llm

problem = """
Design an algorithm to find the kth largest element in an unsorted array
with the optimal time complexity. Analyze the time and space complexity
of your solution and explain why it's optimal.
"""
response = chain.invoke({"problem": problem})
print(response.content)

To find the **k-th largest element** in an unsorted array with **optimal time complexity**, we use the **Quickselect algorithm**, which is a selection algorithm based on the divide-and-conquer strategy of Quicksort.

---

## ✅ Algorithm: **Quickselect**

### **Intuition**
Quickselect is similar to Quicksort but only recurses into one partition (the one containing the desired element), making it more efficient for selection problems.

We want the **k-th largest** element. This is equivalent to finding the **(n - k + 1)-th smallest** element, where `n` is the array length. For simplicity, we can **redefine k** as the index in a 0-based sorted array:  
> **k-th largest = (n - k)-th smallest** (0-indexed)

Alternatively, we can modify the partitioning to find the k-th largest directly by reversing the comparison.

---

### **Algorithm Steps**

1. **Input**: Array `arr` of length `n`, integer `k` (1-indexed k-th largest)
2. **Convert**: Let `target_index = n - k` (we want the element that w

### Prompts and templates

In [9]:
from langchain_core.prompts import ChatPromptTemplate


chat = Config().new_openai_like()

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English to French translator."),
        ("user", "Translate this to French: {text}"),
    ]
)

formatted_messages = template.format_messages(text="Hello, how are you?")
result = chat.invoke(formatted_messages)
print(result.content)

Bonjour, comment allez-vous ?


### LangChain Expression Language (LCEL)

#### Simple workflows with LCEL

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Create components
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")

llm = Config().new_openai_like()

output_parser = StrOutputParser()

# Chain them together using LCEL
chain = prompt | llm | output_parser

# Use the chain
result = chain.invoke({"topic": "programming"})
print(result)

Sure! Here's a classic one for you:

**Why do programmers always mix up Halloween and Christmas?**

Because  
**Oct 31 == Dec 25** 🎃🎄

*Explanation:*  
In octal (base-8), "31" equals 25 in decimal (base-10):  
₈31 = 3×8 + 1 = 25₁₀

So… it’s not a mix-up — it’s *correct*… if you’re a programmer. 😄


#### Complex chain example

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


chat = Config().new_openai_like()

# First chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
story_chain = story_prompt | chat | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analyze the following story's mood:\n{story}"
)
analysis_chain = analysis_prompt | chat | StrOutputParser()

output_prompt = PromptTemplate.from_template(
    "Here's the story: \n{story}\n\nHere's the mood: \n{mood}"
)
# Combine chains
story_with_analysis = story_chain | analysis_chain

# Run the combined chain
result = story_with_analysis.invoke({"topic": "a rainy day"})
print(result)

This story’s mood is **quietly hopeful, tenderly melancholic, and ultimately warm**—a delicate balance between sorrow and renewal, isolation and connection. It unfolds like a gentle rain: slow, immersive, and emotionally resonant.

### Breakdown of the Mood:

#### 1. **Melancholy and Solitude (Opening)**
The story begins with a subdued, almost mournful tone:
- “*The rain came like a whispered secret*” — evokes intimacy but also secrecy, something unspoken, hidden.
- “*Turned the world gray*” — visual and emotional desaturation.
- The man in the long coat “*didn’t look up. No one did.*” — emphasizes emotional detachment, urban alienation.
- Clara sits alone, touching a “*forgotten poetry collection*” — suggesting neglect, loneliness, perhaps grief.

This establishes a mood of quiet sadness, a world washed in gray, both literally and emotionally.

#### 2. **Whispered Wonder and Innocence (The Girl’s Arrival)**
The mood shifts subtly but decisively with the girl’s appearance:
- “*A small 

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


chat = Config().new_openai_like()

# First chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
story_chain = story_prompt | chat | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analyze the following story's mood:\n{story}"
)
analysis_chain = analysis_prompt | chat | StrOutputParser()

output_prompt = PromptTemplate.from_template(
    "Here's the story: \n{story}\n\nHere's the mood: \n{mood}"
)

# Using RunnablePassthrough.assign to preserve data
enhanced_chain = RunnablePassthrough.assign(
    story=story_chain  # Add 'story' key with generated content
).assign(
    analysis=analysis_chain  # Add 'analysis' key with analysis of the story
)

result = enhanced_chain.invoke({"topic": "a rainy day"})
print(result.keys())  # Output: dict_keys(['topic', 'story', 'analysis'])
# dict_keys(['topic', 'story', 'analysis'])

dict_keys(['topic', 'story', 'analysis'])


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

chat = Config().new_openai_like()

# First chain generates a story
story_prompt = PromptTemplate.from_template("Write a short story about {topic}")
story_chain = story_prompt | chat | StrOutputParser()

# Second chain analyzes the story
analysis_prompt = PromptTemplate.from_template(
    "Analyze the following story's mood:\n{story}"
)
analysis_chain = analysis_prompt | chat | StrOutputParser()

output_prompt = PromptTemplate.from_template(
    "Here's the story: \n{story}\n\nHere's the mood: \n{mood}"
)

# Alternative approach using dictionary construction
manual_chain = (
    RunnablePassthrough()  # Pass through input
    | {
        "story": story_chain,  # Add story result
        "topic": itemgetter("topic"),  # Preserve original topic
    }
    | RunnablePassthrough().assign(  # Add analysis based on story
        analysis=analysis_chain
    )
)
result = manual_chain.invoke({"topic": "a rainy day"})
print(result.keys())  # Output: dict_keys(['story', 'topic', 'analysis'])

dict_keys(['story', 'topic', 'analysis'])


## Running local models

### Getting started with Ollama

In [ ]:
!bash ollama.sh up

In [ ]:
!bash ollama.sh run deepseek-r1:1.5b

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize Ollama with your chosen model
local_llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0,
    # 将 base_url 的地址部分替换为上一步打印出来的地址。
    base_url="http://172.17.0.4:11434",
)
# Create an LCEL chain using the local model
prompt = PromptTemplate.from_template("Explain {concept} in simple terms")
local_chain = prompt | local_llm | StrOutputParser()
# Use the chain with your local model
result = local_chain.invoke({"concept": "quantum computing"})
print(result)

In [ ]:
!bash ollama.sh down

### Working with Hugging Face models locally

In [ ]:
%uv pip install langchain-huggingface transformers torch==2.8.0

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline

# Create a pipeline with a small model:
llm = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
    ),
)
chat_model = ChatHuggingFace(llm=llm)
# Use it like any other LangChain LLM
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="Explain the concept of machine learning in simple terms"),
]
ai_msg = chat_model.invoke(messages)
print(ai_msg.content)

### Tips for local models

In [ ]:
import time


def safe_model_call(llm, prompt, max_retries=2):
    """Safely call a local model with retry logic and graceful failure"""
    retries = 0
    while retries <= max_retries:
        try:
            return llm.invoke(prompt)
        except RuntimeError as e:
            # Common error with local models when running out of VRAM
            if "CUDA out of memory" in str(e):
                print(
                    f"GPU memory error, waiting and retrying ({retries+1}/{max_retries+1})"
                )
                time.sleep(2)  # Give system time to free resources
                retries += 1
            else:
                print(f"Runtime error: {e}")
                return "An error occurred while processing your request."
        except Exception as e:
            print(f"Unexpected error calling model: {e}")
            return "An error occurred while processing your request."
    # If we exhausted retries
    return "Model is currently experiencing high load. Please try again later."


# Use the safety wrapper in your LCEL chain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

llm = Config().new_openai_like()

prompt = PromptTemplate.from_template("Explain {concept} in simple terms")
safe_llm = RunnableLambda(lambda x: safe_model_call(llm, x))
safe_chain = prompt | safe_llm
response = safe_chain.invoke({"concept": "quantum computing"})
print(response)

## Multimodal AI applications
### Text-to-image
#### Using DALL-E through OpenAI
TODO：找到或实现支持 langchain 接口的千义同问文生图模型（候选 qwen-image-plus）。

### Image understanding
#### Using Qwen-VL (similar to Gemini 1.5 Pro)

In [ ]:
import base64

from langchain_core.messages.human import HumanMessage

with open("static/stable-diffusion.png", "rb") as image_file:
    image_bytes = image_file.read()
    base64_bytes = base64.b64encode(image_bytes).decode("utf-8")


prompt = [
    {"type": "text", "text": "Describe the image: "},
    {
        "type": "image_url",
        "image_url": {"url": f"data:image/jpeg;base64,{base64_bytes}"},
    },
]


# initialize OpenAI-like model
llm = Config().new_openai_like_vl()
response = llm.invoke([HumanMessage(content=prompt)])
print(response.content)

In [ ]:
import base64

from langchain_core.messages import HumanMessage
from utils import Config


def analyze_image(image_path: str, question: str) -> str:
    # chat = ChatOpenAI(model="gpt-4o-mini", max_tokens=256)
    chat = Config().new_openai_like_vl(max_tokens=256)

    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
        base64_bytes = base64.b64encode(image_bytes).decode("utf-8")

    message = HumanMessage(
        content=[
            {"type": "text", "text": question},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_bytes}",
                    "detail": "auto",
                },
            },
        ]
    )

    response = chat.invoke([message])
    return response.content


# Example usage
image_path = "static/skyscrapers.png"
questions = [
    "What objects do you see in this image?",
    "What is the overall mood or atmosphere?",
    "Are there any people in the image?",
]

for question in questions:
    print(f"\nQ: {question}")
    print(f"A: {analyze_image(image_path, question)}")